In [ ]:
import numpy as np
from scipy.io import loadmat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
import tensorflow as tf
import h5py

2025-06-17 14:06:37.945864: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 14:06:40.318285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow
tensorflow.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2025-06-17 14:06:52.828125: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False

In [3]:
# %whos
# Load the .mat file
file = h5py.File('Train.mat', 'r')

# Extract datasets
XTrain = file['X_normalized'][:]  # Shape: (200, 200,200, 1, n)
YTrain = file['Y_img'][:]  # Shape: (n, 2)

XTrain_permuted=np.transpose(XTrain,(0,3,4,2,1))
YTrain_permuted=np.transpose(YTrain,(1,0))

print(XTrain_permuted.shape)
print(YTrain_permuted.shape)

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
#import tensorflow_hub as hub

import datetime

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the TensorBoard extension
data = XTrain_permuted
data = tf.squeeze(data, axis=4)
print(data.shape)

# Create the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.01,
    height_shift_range=0.01,
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:
# Ensure data and YTrain_permuted are numpy arrays
data = np.array(data)
YTrain_permuted = np.array(YTrain_permuted)

# Define the split ratio
split_ratio = 0.2
split_index = int((1 - split_ratio) * len(data))

# Shuffling the dataset before splitting
indices = np.random.permutation(len(data))
train_indices = indices[:split_index]
val_indices = indices[split_index:]

train_data = data[train_indices]
train_labels = YTrain_permuted[train_indices]
val_data = data[val_indices]
val_labels = YTrain_permuted[val_indices]

In [ ]:
import numpy as np
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Resizing,GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Load pre-trained ConvNeXtTiny model without the top layer
base_model = ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable=True

# Create full model for fine-tunin
model = Sequential()
model.add(Resizing(224, 224, input_shape=(100,100,3)))
model.add(base_model)
model.add(GlobalAveragePooling2D())


model.add(BatchNormalization())
model.add(Dense(128,activation='relu', kernel_regularizer=l2(1e-6)))
model.add(Dropout(0.15))
model.add(Dense(1,kernel_regularizer=l2(1e-6)))


# Use a small learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)


lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Compile the model
model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

# Setup early stopping to prevent over-fitting.
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



In [ ]:
# Fit the model using the data augmentation generator
import time
start_time=time.time()
history = model.fit(
    datagen.flow(train_data, train_labels, batch_size=32),
    epochs=100,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping,lr_reducer]
)
end_time=time.time()
print(f"Training time: {end_time-start_time:.2f} seconds")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss and accuracy values
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot loss on the left y-axis
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:blue')
if 'val_loss' in history.history:
    ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:cyan')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.legend(loc='upper left')
ax1.grid(True)

# Title and layout
plt.title('Model Loss and Accuracy')
fig.tight_layout()

# Save the plot as an image file
plt.savefig('Convtiny.png')

# Show the plot
plt.show()


In [ ]:
history_data = history.history
import scipy.io
scipy.io.savemat('Convtiny_history.mat', history_data)

model.save('Convtiny.keras')